# CS 534: Spring 2018
# Midterm Exam

This exam has 7 questions for a total of 130 points.

Name: Simon Swenson

| Question | Points | Score |
| --------:| ------:| -----:|
|        1 |     30 |       |
|        2 |     20 |       |
|        3 |     10 |       |
|        4 |     20 |       |
|        5 |     15 |       |
|        6 |     15 |       |
|        7 |     20 |       |
|   Total: |    130 |       |

Be sure to show all work. Attach extra sheets as necessary. Collaboration is allowed for only 30 points. Please indicate your collaborator and where you collaborate. 

1: (30 Points) Design a gradient descent algorithm to fit a logistic regression with the following training set (show the derivatives for each component, no vector notation allowed):

| x1 | x2 | x3 | y |
| --:| --:| --:| -:|
|  0 |  1 |  0 | 1 |
|  1 |  0 |  0 | 1 |
|  0 |  0 |  1 | 1 |
|  1 |  1 |  1 | 0 |
|  1 |  1 |  0 | 0 |

Recall that the logistic function is defined as:

$$
f(x) = \frac{e^{wx + b}}{1 + e^{wx + b}} = \frac{1}{1 + \frac{1}{e^{wx + b}}}
$$

Recall the formulation for log likelihood:

$$
\begin{align}
    log L(w, b | x) &= \frac {\sum_{i = 1}^n y_i log(P(y_i | x_i, w, b)) + (1 - y_i) log(1 - P(y_i | x_i, w, b))} {N} \\
                 &= \frac {\sum_{i = 1}^n y_i log(\frac {1} {1 + \frac {1} {e^{wx + b}}}) + (1 - y_i) log(1 - \frac {1} {1 + \frac {1} {e^{wx + b}}})} {N} \\
                 &= \frac {\sum_{i = 1}^n y_i log(\frac {1} {1 + \frac {1} {e^{w_1x_1 + w_2x_2 + w_3x_3 + b}}}) + (1 - y_i) log(1 - \frac {1} {1 + \frac {1} {e^{w_1x_1 + w_2x_2 + w_3x_3 + b}}})} {N}
\end{align}
$$

Then the gradient will be a vector in 4-space with partials for each $ w_i $ and $ b $ term:

$$
\begin{align}
    \frac{\partial log L} {\partial w_1} &= \sum_{i = 1}^n x_{i(1)} (y_i - f(x_i)) \\
    \frac{\partial log L} {\partial w_2} &= \sum_{i = 1}^n x_{i(2)} (y_i - f(x_i)) \\
    \frac{\partial log L} {\partial w_3} &= \sum_{i = 1}^n x_{i(3)} (y_i - f(x_i)) \\
    \frac{\partial log L} {\partial b} &= \sum_{i = 1}^n y_i - f(x_i)
\end{align}
$$

(I don't know if you wanted us to compute the derivatives by hand, but I used Wolfram Alpha to help me out a bit with the calculus.)

Note that we do not normalize the gradient.

Then the algorithm is:

```
delta = infinity
last-cost = -infinity
while delta > stopping-delta:
    dw1 = 0
    dw2 = 0
    dw3 = 0
    db = 0
    for each data-point in data:
        # I use base-1 indexing to be consistent with the weight/x labels in the math above
        dw1 += data-point[1] * (classes[data-point] - logistic(data-point, w1, w2, w3, b))
        dw2 += data-point[2] * (classes[data-point] - logistic(data-point, w1, w2, w3, b))
        dw3 += data-point[3] * (classes[data-point] - logistic(data-point, w1, w2, w3, b))
        db += class[data-point] - logistic(data-point, w1, w2, w3, b)
    # Since we are to maximize likelihood, let's add the gradient
    w1 += dw1
    w2 += dw2
    w3 += dw3
    b += db
    delta = likelihood(data, classes, w1, w2, w3, b) - last-cost
    last-cost = likelihood(data, classes, w1, w2, w3, b)
```

In [1]:
# Python code
import numpy as np
import sys
def logistic(data, weights, bias):
    return 1/(1 + np.exp(-(data[0]*weights[0] + data[1]*weights[1] + data[2]*weights[2] + bias)))

def likelihood(data, classes, weights, bias):
    result = 1.0
    for data_cur, class_cur in zip(data, classes):
        logi = logistic(data_cur, weights, bias)
        result *= logi ** class_cur + (1 - logi) ** (1 - class_cur)
    return result
    
def gradient_descent(data, classes, weights, bias, speed = 0.01, stopping_delta = 0.01):
    delta = sys.float_info.max
    last_cost = sys.float_info.max
    while delta > stopping_delta:
        dw = [0] * 3
        db = 0
        for data_cur, class_cur in zip(data, classes):
            diff = class_cur - logistic(data_cur, weights, bias)
            dw[0] += data_cur[0] * diff
            dw[1] += data_cur[1] * diff
            dw[2] += data_cur[2] * diff
            db += diff
        weights[0] += dw[0] * speed
        weights[1] += dw[1] * speed
        weights[2] += dw[2] * speed
        bias += db * speed
        cost = -likelihood(data, classes, weights, bias)
        delta = abs(cost - last_cost)
        last_cost = cost
    return weights, bias

2: (20 points) Change the formulation of the logistic regression and its gradient descendent algorithm to handle the importance (IP) of the examples (no vector notation is allowed, show the algorithm).

| x1 | x2 | x3 | y | IP |
| --:| --:| --:| -:| --:|
|  0 |  1 |  0 | 1 |  2 |
|  1 |  0 |  0 | 1 |  2 |
|  0 |  0 |  1 | 1 |  2 |
|  1 |  1 |  1 | 0 |  3 |
|  1 |  1 |  0 | 0 |  3 |

Change the likelihood function to:

$$
log L(w, b | x) = \frac {\sum_{i = 1}^n ip_i (y_i log(\frac {1} {1 + \frac {1} {e^{w_1x_1 + w_2x_2 + w_3x_3 + b}}}) + (1 - y_i) log(1 - \frac {1} {1 + \frac {1} {e^{w_1x_1 + w_2x_2 + w_3x_3 + b}}}))} {N}
$$

Note N must also be changed:

$$
N = \sum_{i = 1}^n ip_i
$$

Then the gradient becomes:

$$
\begin{align}
    \frac{\partial log L} {\partial w_1} &= \sum_{i = 1}^n ip_i x_{i(1)} (y_i - f(x_i)) \\
    \frac{\partial log L} {\partial w_2} &= \sum_{i = 1}^n ip_i x_{i(2)} (y_i - f(x_i)) \\
    \frac{\partial log L} {\partial w_3} &= \sum_{i = 1}^n ip_i x_{i(3)} (y_i - f(x_i)) \\
    \frac{\partial log L} {\partial b} &= \sum_{i = 1}^n ip_i (y_i - f(x_i))
\end{align}
$$

In [2]:
# Python code
import numpy as np
import sys
def logistic_imp(data, weights, bias):
    return 1/(1 + np.exp(-(data[0]*weights[0] + data[1]*weights[1] + data[2]*weights[2] + bias)))

def likelihood_imp(data, classes, importance, weights, bias):
    result = 1.0
    for data_cur, class_cur, importance_cur in zip(data, classes, importance):
        logi = logistic_imp(data_cur, weights, bias)
        result *= importance_cur * (logi ** class_cur + (1 - logi) ** (1 - class_cur))
    return result
    
def gradient_descent_imp(data, classes, importance, weights, bias, speed = 0.01, stopping_delta = 0.01):
    delta = sys.float_info.max
    last_cost = sys.float_info.max
    while delta > stopping_delta:
        dw = [0] * 3
        db = 0
        for data_cur, class_cur, importance_cur in zip(data, classes, importance):
            diff = importance_cur * (class_cur - logistic_imp(data_cur, weights, bias))
            dw[0] += data_cur[0] * diff
            dw[1] += data_cur[1] * diff
            dw[2] += data_cur[2] * diff
            db += diff
        weights[0] += dw[0] * speed
        weights[1] += dw[1] * speed
        weights[2] += dw[2] * speed
        bias += db * speed
        cost = -likelihood_imp(data, classes, importance, weights, bias)
        delta = abs(cost - last_cost)
        last_cost = cost
        #print(cost)
    return weights, bias

# I think I maybe generalized more than you wanted for Q1 and Q2. I have the code calculating each gradient coefficient. The example you gave just had the coefficients pre-calculated by hand, though.

3: (10 points) Show the results for both points 1 and point 2 for 3 different initial points in input to the gradient descendent algorithm.

In [3]:
data = [[0, 1, 0], [1, 0, 0], [0, 0, 1], [1, 1, 1], [1, 1, 0]]
classes = [1, 1, 1, 0, 0]
importance = [2, 2, 2, 3, 3]

In [4]:
gradient_descent(data, classes, [0, 0, 0], 0, 1)

([-6.6615549760803745, -6.6615549760803745, -2.421964436360076],
 10.122710661461)

In [5]:
gradient_descent(data, classes, [3, 3, 3], -3, 1)

([-6.658291394397741, -6.658291394397741, -2.41657273234312],
 10.117572435571061)

In [6]:
gradient_descent(data, classes, [-1, 1, -2], 14, 1)

([-6.781998426400366, -6.596651299776646, -3.882604932995159],
 10.209943432929636)

In [7]:
gradient_descent_imp(data, classes, importance, [0, 0, 0], 0, 1)

([-11.917666892123211, -11.917666892123211, -3.264227388327358],
 17.822948536586665)

In [8]:
gradient_descent_imp(data, classes, importance, [3, 3, 3], -3, 1)

([-11.916869544241836, -11.916869544241836, -3.3024333320435963],
 17.822457344183352)

In [9]:
gradient_descent_imp(data, classes, importance, [-1, 1, -2], 14, 1)

([-11.888144539940846, -11.888143752924751, -4.480750513872881],
 17.79204647078796)

4: (20 points) By using a binary classifier (logistic regression or SVM) please implement in python the 2 different procedure (one vs. one, one vs. other) to handle the problem of more than two different classes (use the iris dataset).

In [10]:
import pandas as pd
from sklearn import utils
from sklearn import preprocessing as pp
from sklearn import svm

data = utils.shuffle(np.array(pd.read_csv('iris1.csv')), random_state = 42)
X = data[:, :4]
y = data[:, 4].flatten()

le = pp.LabelEncoder()
y = le.fit_transform(y)
classoh = pp.OneHotEncoder()
y = classoh.fit_transform(y.reshape(150, 1))

In [11]:
# one-vs-others
one_vs_others_classifiers = [svm.LinearSVC(), svm.LinearSVC(), svm.LinearSVC()]
for i in range(3):
    yy = y[:, i].toarray().flatten()
    # Split between training and test
    X_tr = X[:len(X) * 9 // 10, :]
    X_te = X[len(X) * 9 // 10:, :]
    y_tr = yy[:len(X) * 9 // 10]
    y_te = yy[len(X) * 9 // 10:]
    one_vs_others_classifiers[i].fit(X_tr, y_tr)
    classes = one_vs_others_classifiers[i].predict(X_te)
    accur = np.sum([1 if y_te_cur == class_cur else 0 for y_te_cur, class_cur in zip(y_te, classes)])/len(y_te)
    print(accur)

1.0
0.5333333333333333
1.0


In [12]:
# one-vs-one
one_vs_one_classifiers = {}
for i in range(2):
    for j in range(i + 1, 3):
        XX = []
        yy = []
        for X_cur, y_cur in zip(X, y.toarray()):
            if y_cur[i] == 1:
                XX.append(X_cur)
                yy.append(1)
            elif y_cur[j] == 1:
                XX.append(X_cur)
                yy.append(0)
        XX = np.array(XX)
        yy = np.array(yy)
        X_tr = XX[:len(XX) * 9 // 10, :]
        X_te = XX[len(XX) * 9 // 10:, :]
        y_tr = yy[:len(XX) * 9 // 10]
        y_te = yy[len(XX) * 9 // 10:]
        cur_classifier = svm.LinearSVC()
        cur_classifier.fit(X_tr, y_tr)
        classes = cur_classifier.predict(X_te)
        accur = np.sum([1 if y_te_cur == class_cur else 0 for y_te_cur, class_cur in zip(y_te, classes)])/len(y_te)
        print(accur)
        one_vs_one_classifiers[(i, j)] = cur_classifier

1.0
1.0
1.0


5: (15 points) Implement the Naïve Bayes classifier, use the training set in point 1 and test it with the following dataset

| x1 | x2 | x3 | y |
| --:| --:| --:| -:|
|  1 |  0 |  0 | 0 |
|  1 |  0 |  1 | 1 |

In [13]:
X_tr = np.array([[0, 1, 0], [1, 0, 0], [0, 0, 1], [1, 1, 1], [1, 1, 0]])
y_tr = np.array([1, 1, 1, 0, 0])

# For each class, for each feature, for each category of that feature,
# we must calculate P(cat | class). Also need P(class)

# Creates a new several-layers deep dictionary that matches class + feature index + feature
# to a probability and class to a probability. Initializes all leaf nodes to 0.
# The structure is:
# feature_instances
#     classes
#         features
#             feature category probability (likelihood), initialized to 0.
# class_instances
#     class probability (num class/total instances), initialized to 0.
def empty_NBC(feature_domain, class_domain):
    result = {'feature_instances': {}, 'class_instances': {}}
    for class_domain_cur in class_domain:
        result['feature_instances'][class_domain_cur] = {}
        result['class_instances'][class_domain_cur] = 0
        for feature_domain_idx in range(len(feature_domain)):
            result['feature_instances'][class_domain_cur][feature_domain_idx] = {}
            for feature in feature_domain[feature_domain_idx]:
                result['feature_instances'][class_domain_cur][feature_domain_idx][feature] = 0
    return result

# Fills an assumed empty dictionary, first with the number of occurrances of each
# situation, then normalizes/divides by the number of occurrances for the corresponding
# class.
def fill_NBC(NBC, X_tr, y_tr):
    total = len(y_tr)
    for X_cur, y_cur in zip(X_tr, y_tr):
        # Count the number of occurrances of each class
        NBC['class_instances'][y_cur] +=1
        for feature_idx in range(len(X_cur)):
            feature = X_cur[feature_idx]
            # Count the number of occurrances of each situation
            NBC['feature_instances'][y_cur][feature_idx][feature] += 1
    
    # Divide the number of each situation by the corresponding class, calculating likelihood
    for i in NBC['feature_instances']:
        for j in NBC['feature_instances'][i]:
            for k in NBC['feature_instances'][i][j]:
                # Divide by the number of instances of the corresponding class
                NBC['feature_instances'][i][j][k] /= NBC['class_instances'][i]
                
    # P(c)
    total = len(y_tr)
    for i in NBC['class_instances']:
        NBC['class_instances'][i] /= total
                
# Do the classification, comparing the output probabilities of the model for each class and
# picking the one with the highest probability.
def classify(NBC, X_te):
    best_prediction = -1.0
    best_prediction_class = ''
    for cclass in NBC['class_instances']:
        # Calculate the probability using Bayes's Theorem
        cur_prediction = NBC['class_instances'][cclass]
        for col_idx in range(len(X_te)):
            col = X_te[col_idx]
            cur_prediction *= NBC['feature_instances'][cclass][col_idx][col]
        # Only keep the best probability/prediction
        if cur_prediction > best_prediction:
            best_prediction = cur_prediction
            best_prediction_class = cclass
    return best_prediction_class, best_prediction
    
NBC = empty_NBC([[0, 1], [0, 1], [0, 1]], [0, 1])
fill_NBC(NBC, X_tr, y_tr)

X_te = np.array([[1, 0, 0], [1, 0, 1]])
y_te = np.array([0, 1])

for point in X_te:
    print('classification for', point, ":", classify(NBC, point.flatten()))

classification for [1 0 0] : (1, 0.08888888888888886)
classification for [1 0 1] : (1, 0.04444444444444443)


6: (15 points) Implement the K-nearest neighbor classifier with the same training and test of the point 5.

In [14]:
def classify_KNN(X_tr, y_tr, X_te, K):
    class_distances = []
    for X_tr_cur, y_tr_cur in zip(X_tr, y_tr):
        distance = 0
        # I use euclidian (l2) distance. You can easily do something else here.
        for X_tr_feature, X_te_feature in zip(X_tr_cur, X_te):
            distance += (X_te_feature - X_tr_feature) ** 2
        distance = distance ** 0.5
        # Building up a tuple that contains the distance calculation and the corresponding class
        class_distances.append((y_tr_cur, distance))
    # Sort the tuples by distance.
    class_distances = sorted(class_distances, key = lambda cur: cur[1])
    
    # Find the consensus of various classes
    consensus = {}
    for i in range(min(len(class_distances), K)):
        if not class_distances[i][0] in consensus:
            consensus[class_distances[i][0]] = 1
        else:
            consensus[class_distances[i][0]] += 1
            
    # Find the highest consensus
    best_key_num = 0
    best_key = ''
    for key in consensus:
        if consensus[key] > best_key_num:
            best_key_num = consensus[key]
            best_key = key
    return best_key, best_key_num
    
classify_KNN(X_tr, y_tr, X_te[0], 3)

(1, 2)

7: (20 points) Create examples to explain the property an the importance of the following kernels:

- https://en.wikipedia.org/wiki/Graph_kernel
- https://en.wikipedia.org/wiki/String_kernel
- https://en.wikipedia.org/wiki/Polynomial_kernel, in this case make a comparison with another procedure that in not based on kernels.

Suppose you were tasked creating a system that will classify systems of roads into several categories (city, suburban, country, dirt, etc.), just given their intersections (the nodes) and distance to each intersection (the connection). These graphs are not bounded in any way. Can a road system have 100 roads and 200 connections? What about 1000 roads and 10,000 connections? It is not a well-defined feature set. However, we can use the graph kernel to compare these graphs anyway, even if they are completely different. Using the kernel process, we can create a feature set with length equal to the number of data points, where each feature is the graph kernel of a point in the data set and the current graph. This allows us to transform the data into something that is easier to work with.

Words and sentences, just like the graph above, contain an indefinate number of characters. It is possible to create a run-on sentence that goes on virtually forever. We cannot have a feature space with an indefinate number of features. In addition, sentences like, "I love machine learning!" and "Machine learning is great!" would be represented totally different in a naive approach, where each character is represented as a feature. Since the phrase "machine learning" appears in different positions in the sentences, the representation of both in that feature space doesn't encode that similarity at all. Thus, we turn to a kernel function to compare the strings. This allows us to create another feature space similar to above, where each feature is the string kernel applied to a particular point in the data set and the current point.

If we have a nonlinear dataset, the polynomial kernel can be very useful. It allows us to transform the data using combinations of the features like $ x_1^2, x_1x_2,etc $. Recall the distance transformation, which can similarly classify nonlinear data. The distance transformation converts each feature to be the distance of that point to another specific point in the training data. Thus, the number of features will be proportional to the number of data points in the training data. However, with a polynomial kernel, the number of features will be proportional to the number of original features (on the order of number-of-features^n, where n is the number of degrees). Thus, we may want to use one or the other depending on if the data set has few features and many data points or many features and few data points. In addition, testing may be needed to find out the best degree of the polynomial classifier. Either way, the transformations allow us to classify nonlinear data.